### Table of Contents
1. Advanced DataFrame Manipulation
2. Grouping and Aggregating Data
3. Merging and Joining DataFrames
4. Applying Functions with `apply()`, `map()`, `applymap()`
5. Working with Time Series Data
6. Pivot Tables and Crosstabulations
7. Handling Missing Data
8. Performance Optimization in Pandas

---


### 1. **Advanced DataFrame Manipulation**

Pandas provides powerful data manipulation capabilities. You can perform operations on specific columns, slice data, and filter rows based on multiple conditions.

#### Filtering Rows Based on Multiple Conditions

In [42]:
import pandas as pd

# Sample DataFrame
data = {'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve', 'Jan', 'Anna'],
        'Age': [24, 30, 35, 29, 22, 22, 25],
        'City': ['NY', 'SF', 'LA', 'SF', 'LA', 'LA', 'NY'],
        'Salary': [70000, 80000, 120000, 110000, 90000, 90098, 200000]}
df = pd.DataFrame(data)

# Filter based on multiple conditions
filtered_df = df[(df['Age'] > 25) & (df['City'] == 'SF')] # note the element-wise and (&), elemnet-wise or (|)
print(filtered_df)

    Name  Age City  Salary
1    Bob   30   SF   80000
3  David   29   SF  110000


#### Modifying Columns with Conditions

You can modify values in a column based on a condition.

In [43]:
df['High_Earner'] = df['Salary'].apply(lambda x: 'Yes' if x > 100000 else 'No') # note lambda as anonymous function
print(df)

      Name  Age City  Salary High_Earner
0    Alice   24   NY   70000          No
1      Bob   30   SF   80000          No
2  Charlie   35   LA  120000         Yes
3    David   29   SF  110000         Yes
4      Eve   22   LA   90000          No
5      Jan   22   LA   90098          No
6     Anna   25   NY  200000         Yes


### 2. **Grouping and Aggregating Data**

Grouping data is a common task, especially for summarizing information. You can use `groupby()` for splitting the data into groups and applying aggregate functions.


In [25]:
# Group by 'City' and aggregate the average salary and average age
grouped = df.groupby('City').agg({'Salary': 'mean', 'Age': 'max'})
print(grouped)

        Salary  Age
City               
LA    105000.0   35
NY     70000.0   24
SF     95000.0   30


#### Custom Aggregations with `apply()`

In [5]:
def salary_range(series):
    return series.max() - series.min()

grouped_custom = df.groupby('City').agg({'Salary': salary_range})
print(grouped_custom)

      Salary
City        
LA     30000
NY         0
SF     30000


### 3. **Merging and Joining DataFrames**

You can merge or join multiple DataFrames based on keys (like SQL joins). Pandas provides `merge()`, `join()`, and `concat()` to handle these operations.

#### Example of `merge()`

In [27]:
# Two DataFrames
df1 = pd.DataFrame({'Name': ['Alice', 'Bob', 'Charlie'],
                    'Age': [24, 30, 35]})

df2 = pd.DataFrame({'Name': ['Alice', 'Bob', 'Charlie', 'David'],
                    'Salary': [70000, 80000, 120000, 100000]})

# Inner join on 'Name'
merged_df = pd.merge(df1, df2, on='Name', how='inner') # combines rows from two tables (or DataFrames) based on a matching condition (intersection)
#merged_df = pd.merge(df1, df2, on='Name', how='outer') # combines rows from two tables (or DataFrames) (union)

print(merged_df)

      Name  Age  Salary
0    Alice   24   70000
1      Bob   30   80000
2  Charlie   35  120000


#### Example of `concat()`

In [7]:
# Concatenating vertically
df_concat = pd.concat([df1, df2], axis=0, ignore_index=True)
print(df_concat)

      Name   Age    Salary
0    Alice  24.0       NaN
1      Bob  30.0       NaN
2  Charlie  35.0       NaN
3    Alice   NaN   70000.0
4      Bob   NaN   80000.0
5  Charlie   NaN  120000.0
6    David   NaN  100000.0


### 4. **Applying Functions with `apply()`, `map()`, and `applymap()`**

- `apply()` is used to apply a function to each column or row.
- `map()` is used for element-wise transformations on a Series.
- `applymap()` is used for element-wise transformations on a DataFrame.

#### `apply()`


In [8]:
# Apply a function to a column
df['Age_Squared'] = df['Age'].apply(lambda x: x ** 2)
print(df)

      Name  Age City  Salary High_Earner  Age_Squared
0    Alice   24   NY   70000          No          576
1      Bob   30   SF   80000          No          900
2  Charlie   35   LA  120000         Yes         1225
3    David   29   SF  110000         Yes          841
4      Eve   22   LA   90000          No          484


#### `map()`


In [9]:
# Map values in a Series
df['City_Code'] = df['City'].map({'NY': 1, 'SF': 2, 'LA': 3})
print(df)

      Name  Age City  Salary High_Earner  Age_Squared  City_Code
0    Alice   24   NY   70000          No          576          1
1      Bob   30   SF   80000          No          900          2
2  Charlie   35   LA  120000         Yes         1225          3
3    David   29   SF  110000         Yes          841          2
4      Eve   22   LA   90000          No          484          3


#### `applymap()`


In [29]:
# Apply element-wise function on entire DataFrame
df[['Age', 'Salary']].applymap(lambda x: x / 1000)

df[['Age', 'Salary']].map(lambda x: x / 1000)

C:\Users\syousaf\AppData\Local\Temp\ipykernel_27780\780257583.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[['Age', 'Salary']].applymap(lambda x: x / 1000)


,Age,Salary
0,0.024,70.0
1,0.030,80.0
2,0.035,120.0
3,0.029,110.0
4,0.022,90.0


### 5. **Working with Time Series Data**

Pandas has excellent support for time series data. Let’s cover some of the basic operations with dates.

In [46]:
# Creating a DateTime index
dates = pd.date_range('20240101', periods=6)
df_dates = pd.DataFrame({'Sales': [250, 280, 300, 200, 230, 270]}, index=dates)
print(df_dates)

            Sales
2024-01-01    250
2024-01-02    280
2024-01-03    300
2024-01-04    200
2024-01-05    230
2024-01-06    270


### 6. **Pivot Tables and Crosstabulations**

Pivot tables allow you to summarize data similar to Excel’s pivot tables.

#### Creating a Pivot Table


In [44]:
# Pivot table to see average salary by city and high earner status

# values: The column(s) of data you want to aggregate.
# index: The column(s) you want to group by (become the rows of the pivot table).
# columns: The column(s) to be transformed into new columns in the pivot table.
# aggfunc: The aggregation function to be applied, such as sum, mean, count, etc. The default is mean.
    
pivot_table = df.pivot_table(values='Salary', index='City', columns='High_Earner', aggfunc='mean')
print(pivot_table)

High_Earner       No       Yes
City                          
LA           90049.0  120000.0
NY           70000.0  200000.0
SF           80000.0  110000.0


#### Crosstabulations

Crosstab is similar to a pivot table but focuses on counting occurrences.


In [45]:
# Crosstabulation of High_Earner by City
crosstab = pd.crosstab(df['City'], df['High_Earner'])
print(crosstab)

High_Earner  No  Yes
City                
LA            2    1
NY            1    1
SF            1    1


### 7. **Handling Missing Data**

Handling missing values effectively is crucial for data analysis.

#### Checking for Missing Data

In [17]:
# Check for missing data
print(df.isnull())

    Name    Age   City  Salary  High_Earner  Age_Squared  City_Code
0  False  False  False   False        False        False      False
1  False  False  False   False        False        False      False
2  False  False  False   False        False        False      False
3  False  False  False   False        False        False      False
4  False  False  False   False        False        False      False


#### Filling Missing Values


In [18]:
# Fill missing data with a default value
df.fillna(value={'Salary': 0}, inplace=True)

#### Dropping Missing Data

In [19]:
# Drop rows where any data is missing
df.dropna(how='any', inplace=True)

### 8. **Performance Optimization in Pandas**

Understanding how to optimize performance in Pandas is important.

#### Vectorization

Avoid using loops and try to use vectorized operations where possible. Pandas operations are optimized internally for speed.

In [20]:
# Slow version with loop
df['Salary'] = [x * 2 for x in df['Salary']]

# Fast vectorized version
df['Salary'] = df['Salary'] * 2

#### Using `eval()` and `query()`

Pandas’ `eval()` and `query()` can optimize performance when working with large datasets.

In [22]:
# Using query() for filtering
filtered_df = df.query('Age > 30 and Salary > 100000')
print(filtered_df)

      Name  Age City  Salary High_Earner  Age_Squared  City_Code
2  Charlie   35   LA  480000         Yes         1225          3


### Conclusion

We covered some Pandas techniques that are essential for working with real-world data. Here’s what you learned:
- Advanced data manipulation techniques
- Grouping and aggregating data
- Merging and joining DataFrames
- Applying functions with `apply()`, `map()`, and `applymap()`
- Working with time series data
- Creating pivot tables and cross-tabulations
- Handling missing data effectively
- Optimizing performance with vectorization and other methods